In [ ]:
# import os
# os.listdir(os.curdir)
#import mllib
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.mllib.feature import Word2Vec
from pyspark.mllib.feature import StandardScalerModel
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix, BlockMatrix
from pyspark.mllib.linalg import Matrices
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.clustering import BisectingKMeans, BisectingKMeansModel

# python imports
from math import sqrt
import numpy as np
from numpy import array
import os, csv, sys, time
from random import randint
from itertools import izip, izip_longest

In [5]:
# def get_input(fn):

fn = "cl_tiny.csv"
cur_dir = os.path.abspath(os.curdir)
input_file_path = os.path.normpath(os.path.join(cur_dir, "..", "data", fn))
print(input_file_path)

os.path.isfile(input_file_path)

C:\Users\shane\programming\cs657_mining_massive_datasets\craigslist_clustering\data\cl_tiny.csv


True

In [6]:
# [(postTitle, postingURL, postLocation, time, lat, long, address, dateRetrieved, post_date, ad), ...]
# tiny input has 30 reviews
raw_ads = sc.textFile(input_file_path)
# set = input.take(3)

In [7]:
# [ad0, ad1, ..]
ads_rdd = raw_ads.map(lambda x: str(x.decode('utf-8', 'ignore')))
# processed_rdd = input.map(lambda x: str(x.decode('utf-8', 'ignore'))).map(lambda x: x.split(","))
# processed_rdd.take(2)

In [1]:
# def transpose_rdd()
# tfidf_rdd.flatMap(lambda x: x).take(3)
# flatMap by keeping the column position
flat_rdd = tfidf_rdd.flatMap(lambda row: row.map(lambda col: (col, row.indexOf(col))))
flat_rdd.take(3)
# .map(v => (v._2, v._1)) // key by column position
# .groupByKey.sortByKey   // regroup on column position, thus all elements from the first column will be in the first row
# .map(_._2)              // discard the key, keep only value
# df = rdd.toDF()
# # Grab data from first columns, since it will be transposed to new column headers
# new_header = [i[0] for i in dt.select("_1").rdd.map(tuple).collect()]

# # Remove first column from dataframe
# dt2 = dt.select([c for c in dt.columns if c not in ['_1']])

# # Convert DataFrame to RDD
# rdd = dt2.rdd.map(tuple)

# # Transpose Data
# rddT1 = rdd.zipWithIndex().flatMap(lambda (x,i): [(i,j,e) for (j,e) in enumerate(x)])
# rddT2 = rddT1.map(lambda (i,j,e): (j, (i,e))).groupByKey().sortByKey()
# rddT3 = rddT2.map(lambda (i, x): sorted(list(x), cmp=lambda (i1,e1),(i2,e2) : cmp(i1, i2)))
# rddT4 = rddT3.map(lambda x: map(lambda (i, y): y , x))

# # Convert back to DataFrame (along with header)
# df = rddT4.toDF(new_header)

# return df

NameError: name 'tfidf_rdd' is not defined

In [8]:
def get_tfidf(rdd):
    # While applying HashingTF only needs a single pass to the data, applying IDF needs two passes:
    # First to compute the IDF vector and second to scale the term frequencies by IDF.
    hashingTF = HashingTF()
    tf = hashingTF.transform(rdd)
    tf.cache()
    idf = IDF().fit(tf)
    tfidf = idf.transform(tf)
    # spark.mllib's IDF implementation provides an option for ignoring terms
    # which occur in less than a minimum number of documents.
    # In such cases, the IDF for these terms is set to 0.
    # This feature can be used by passing the minDocFreq value to the IDF constructor.
    idfIgnore = IDF(minDocFreq=1).fit(tf)
    tfidf_rdd = idfIgnore.transform(tf)
    # rdd of SparseVectors [(doc_id_i: {word_id_j: tfidfscore_j, ...}), ... }]
    # or m docs x n counts
    return tfidf_rdd

In [ ]:
def get_svd_U(tfidf_rdd, n_topics=3):
    # distributed matrix
    matrix_rdd = RowMatrix(tfidf_rdd)

    matrix_rdd.numRows
    # matrix_rdd.rows.take(3)
    svd = matrix_rdd.computeSVD(3, computeU=True)
    
    # left singular vectors
    # type = RowMatrix
    svd_u = svd.U
    
    # array of DenseVectors, m_documents x n_topics
    # [[topic_i, ...], ...]
    return svd_u.rows.collect()

In [ ]:
def save_svd_U(svd_U_rdd, fn="svd_u_results.npz"):
    np.savez(fn,np.array(svd_U_rdd))

In [17]:
tfidf_rdd = get_tfidf(ads_rdd)

In [81]:
#     sentence = "aa bb ab" * 10 + "a cd " * 10
#     localDoc = [sentence, sentence]
#     doc = sc.parallelize(localDoc).map(lambda line: line.split(" "))
#     model = Word2Vec().setVectorSize(10).setSeed(42).fit(doc)
# i think it is expecting a list of document lists [[word1, word2,...], ...]
def get_word2vec(rdd):
    word2vec = Word2Vec()
    model = word2vec.fit(ads_rdd)

In [134]:
# k_means
# Build the model (cluster the data)
# kmeans(rdd, k, maxIterations, runs, InitializationMode, seed, initializationSteps, epsilon, initialModel)
def cluster_kmeans_svd(svd_u, k=2, n_iters=10, save_model=False):
    rdd = svd.U.rows
    clusters = KMeans.train(rdd, k, maxIterations=n_iters, initializationMode="random")
    WSSSE = rdd.map(lambda point: error(point)).reduce(lambda x, y: x + y)
    print("Within Set Sum of Squared Error = " + str(WSSSE))
    if save_model:
        save_cluster_model(clusters, fn)
    
    # do i need to keep the predictions with the keys?
    # returns a list of labels
    return rdd.map(lambda x: ((x[0],x[1]), clusters.predict(x)))

In [ ]:
def save_cluster_predictions(labels, fn="test_results.pkl"):

In [111]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [135]:
kmeans_mapped = cluster_kmeans_svd(svd.U)

Within Set Sum of Squared Error = 5.32034738594


In [136]:
np.savez("km_cluster_results.npz", kmeans_mapped.collect())

In [101]:
# Save and load model
def save_cluster_model(clusters, fn="test_model"):
    clusters.save(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")

In [105]:
def load_cluster_model(clusters, fn="test_model"):
    sameModel = KMeansModel.load(sc, fn)

In [ ]:
def get_model(n_topics, cluster_model="kmeans", cluster_params):
    if cluster_model == "gaus":
        gaussian_clustering()
        pass
    elif cluster_model == "kmeans":
        pass
    elif cluster_model == "bimeans":
        pass
    elif cluster_model == "lda":
        pass
    elif cluster_model == "pic":
        pass
    else:
        print("a viable option wasnt chosen")
    
    


In [ ]:
def grid_search(params, model):
    for i in param:
        rdd = get_svd()
        model = get_model()
        y_hats = make_preds()
        results_to_disk(rdd, y)

In [ ]:
def gaussian_clustering(rdd):
    # Build the model (cluster the data)
    gmm = GaussianMixture.train(parsedData, 2)

    # Save and load model
    gmm.save(sc, "target/org/apache/spark/PythonGaussianMixtureExample/GaussianMixtureModel")
    sameModel = GaussianMixtureModel\
        .load(sc, "target/org/apache/spark/PythonGaussianMixtureExample/GaussianMixtureModel")

    # output parameters of model
    for i in range(2):
        print("weight = ", gmm.weights[i], "mu = ", gmm.gaussians[i].mu,
              "sigma = ", gmm.gaussians[i].sigma.toArray())



In [2]:
def PIC_clustering(rdd, k, n_iters):
    # Load and parse the data
    data = sc.textFile("data/mllib/pic_data.txt")
    similarities = data.map(lambda line: tuple([float(x) for x in line.split(' ')]))

    # Cluster the data into two classes using PowerIterationClustering
    model = PowerIterationClustering.train(similarities, 2, 10)

    model.assignments().foreach(lambda x: print(str(x.id) + " -> " + str(x.cluster)))

    # Save and load model
    model.save(sc, "target/org/apache/spark/PythonPowerIterationClusteringExample/PICModel")
    sameModel = Po[werIterationClusteringModel\
        .load(sc, "target/org/apache/spark/PythonPowerIterationClusteringExample/PICModel")

SyntaxError: invalid syntax (<ipython-input-2-9d67d8e19813>, line 9)

In [ ]:
def LDA_clustering(rdd, k, n_iters, doc_con, topic_con):
    # Load and parse the data
    data = sc.textFile("data/mllib/sample_lda_data.txt")
    parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
    # Index documents with unique IDs
    corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()

    # Cluster the documents into three topics using LDA
    ldaModel = LDA.train(corpus, k=3)

    # Output topics. Each is a distribution over words (matching word count vectors)
    print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())
          + " words):")
    topics = ldaModel.topicsMatrix()
    for topic in range(3):
        print("Topic " + str(topic) + ":")
        for word in range(0, ldaModel.vocabSize()):
            print(" " + str(topics[word][topic]))

    # Save and load model
    ldaModel.save(sc, "target/org/apache/spark/PythonLatentDirichletAllocationExample/LDAModel")
    sameModel = LDAModel\
        .load(sc, "target/org/apache/spark/PythonLatentDirichletAllocationExample/LDAModel")

In [ ]:
def bisecting_kmeans_clustering(rdd, step_size, n_iters, batch_frac, reg):
    model = "bimeans"
    # Load and parse the data
    data = sc.textFile("data/mllib/kmeans_data.txt")
    parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

    # Build the model (cluster the data)
    model = BisectingKMeans.train(parsedData, 2, maxIterations=5)

    # Evaluate clustering
    cost = model.computeCost(parsedData)
    print("Bisecting K-means Cost = " + str(cost))

    # Save and load model
    path = "target/org/apache/spark/PythonBisectingKMeansExample/BisectingKMeansModel"
    model.save(sc, path)
    sameModel = BisectingKMeansModel.load(sc, path)


In [ ]:
cluster_params = (
    {"k":[5]},
    {"n_iters": [100],},

    "bimeans": {
                   "step_zize":[0.5, 0.4],
                   "batch_frac": [0.35],
                   "reg": [0.02]},

               "lda":{
    "doc_con":[0/5],
    "topic_con": [0.3]
}

)
